In [1]:
import warnings

import numpy as np
import pandas as pd
from openpyxl import load_workbook

warnings.simplefilter("ignore")

# RID OSPAR Reporting Template

**Note:** Despite the [changes to the programme](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/programme_changes_2017-18.ipynb) for 2017 to 2020, we have decided to continue to submit data to OSPAR using the old divisions (see e-mail from Øyvind received 15.10.2018 at 15.51). The code in this notebook therefore uses the column named `'old_rid_code'` to allocate fluxes to the correct regions.

The final step in the annual RID reporting is to complete and submit an overall summary template to OSPAR. A blank template and guideline documentation were provided by Eva (see e-mail received 04/09/2017 at 13.07) and templates are usually provided each year by Csilla.

**Note:** Tore previously calculated "upper" load estimates by setting LOD values equal to the LOD itself, and "lower" estimates by setting LOD values equal to zero. More recently, we have handled LOD values  explicity, by calculating a single estimate as described [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/rid_data_exploration.ipynb#2.3.-Calculate-loads). We therefore only need to fill-in entires for the "mean" rows.

Most of the information required for the template has already been reported in other tables, especially the following:

 * `Table_3_{year}.docx` <br><br>
 
 * `loads_and_flows_all_sites_{year}.csv` <br><br>
 
 * `concs_and_flows_rid_11-36_{year}.csv`

However, some information is not available from these tables. In previous years Tore has simply left these cells blank and my code here does the same, although in principle we could fill-in the whole thing with a bit more work.

In [2]:
# Set year of interest
year = 2022

## 1. Get summary data

The notebooks `summary_table_{year}.ipynb` calculate summary values for Table 3. Rather than reading these values from Word, it is easier to repeat the code to summarise the raw data again. This is the data that needs writing to the OSPAR template.

### 1.1. Monitored areas

In [3]:
# Read data
in_csv = f"/home/jovyan/shared/common/elveovervakingsprogrammet/results/measured_loads/loads_and_flows_all_sites_{year}.csv"
mon_df = pd.read_csv(in_csv)
del mon_df["new_rid_group"]

# Group by OSPAR region
mon_df1 = mon_df.groupby(["ospar_region", "old_rid_group"]).sum()

# Totals for Norway
mon_df2 = mon_df.groupby("old_rid_group").sum().reset_index()
mon_df2["ospar_region"] = "NORWAY"
mon_df2.set_index(["ospar_region", "old_rid_group"], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split("_")[1] != "Est"]
mon_df = mon_df[cols]
del mon_df["station_id"], mon_df["mean_q_1000m3/day"]

# Convert units
mon_df["Hg_kg"] = mon_df["Hg_kg"] / 1000.0  # kg to tonnes
mon_df["NH4-N_tonnes"] = mon_df["NH4-N_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["NO3-N_tonnes"] = mon_df["NO3-N_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["TOTN_tonnes"] = mon_df["TOTN_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["TOTP_tonnes"] = mon_df["TOTP_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["PO4-P_tonnes"] = mon_df["PO4-P_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["SPM_tonnes"] = mon_df["SPM_tonnes"] / 1000.0  # tonnes to ktonnes

# Units are correct, so remove
mon_df.columns = [i.split("_")[0] for i in mon_df.columns]

mon_df.round(0)

Ag   As   Cd    Cr    Cu   Hg  NH4-N  \
ospar_region        old_rid_group                                          
LOFOTEN-BARENTS SEA rid_108        0.0  1.0  0.0   1.0   6.0  0.0    0.0   
                    rid_11         0.0  1.0  0.0   1.0   2.0  0.0    0.0   
                    rid_36         0.0  1.0  0.0   4.0  11.0  0.0    0.0   
NORTH SEA           rid_108        0.0  1.0  0.0   1.0   8.0  0.0    0.0   
                    rid_11         0.0  0.0  0.0   0.0   1.0  0.0    0.0   
                    rid_36         0.0  1.0  0.0   2.0   8.0  0.0    0.0   
NORWEGIAN SEA2      rid_108        0.0  1.0  0.0   1.0   6.0  0.0    0.0   
                    rid_11         0.0  1.0  0.0   2.0  12.0  0.0    0.0   
                    rid_36         0.0  4.0  0.0  11.0  33.0  0.0    0.0   
SKAGERAK            rid_108        0.0  1.0  0.0   1.0   2.0  0.0    0.0   
                    rid_11         0.0  5.0  0.0   9.0  44.0  0.0    1.0   
                    rid_36         0.0  1.0  0.0   0.0   4.0  0.0    0.0   
NORWAY              rid_108        0.0  4.0  0.0   4.0  23.0  0.0    0.0   
                    rid_11         0.0  7.0  0.0  12.0  59.0  0.0    1.0   
                    rid_36         0.0  7.0  0.0  18.0  57.0  0.0    1.0   

                                   NO3-N    Ni  PO4-P   Pb    SPM      SiO2  \
ospar_region        old_rid_group                                             
LOFOTEN-BARENTS SEA rid_108          0.0   6.0    0.0  0.0   11.0   23826.0   
                    rid_11           0.0   1.0    0.0  0.0  290.0   21939.0   
                    rid_36           1.0  23.0    0.0  0.0   31.0   80506.0   
NORTH SEA           rid_108          2.0   5.0    0.0  2.0   19.0   20485.0   
                    rid_11           0.0   1.0    0.0  0.0    5.0    3757.0   
                    rid_36           3.0   3.0    0.0  3.0   32.0   26216.0   
NORWEGIAN SEA2      rid_108          1.0   4.0    0.0  1.0   24.0   20873.0   
                    rid_11           1.0   5.0    0.0  1.0   28.0   16091.0   
                    rid_36           3.0  24.0    0.0  3.0  149.0   61681.0   
SKAGERAK            rid_108          1.0   2.0    0.0  1.0    9.0    6593.0   
                    rid_11          11.0  20.0    0.0  6.0  229.0  129872.0   
                    rid_36           1.0   2.0    0.0  2.0   11.0   12500.0   
NORWAY              rid_108          5.0  17.0    0.0  4.0   63.0   71777.0   
                    rid_11          12.0  27.0    0.0  7.0  552.0  171660.0   
                    rid_36           7.0  53.0    0.0  8.0  222.0  180904.0   

                                        TOC  TOTN  TOTP     Zn  
ospar_region        old_rid_group                               
LOFOTEN-BARENTS SEA rid_108         28346.0   1.0   0.0    7.0  
                    rid_11          13904.0   1.0   0.0    1.0  
                    rid_36          51708.0   2.0   0.0   16.0  
NORTH SEA           rid_108         22269.0   4.0   0.0   23.0  
                    rid_11           5312.0   1.0   0.0    5.0  
                    rid_36          31497.0   5.0   0.0   37.0  
NORWEGIAN SEA2      rid_108         25265.0   2.0   0.0    9.0  
                    rid_11          16831.0   1.0   0.0   21.0  
                    rid_36          91693.0   8.0   0.0   78.0  
SKAGERAK            rid_108         11867.0   2.0   0.0   15.0  
                    rid_11         152136.0  18.0   0.0  104.0  
                    rid_36          33943.0   2.0   0.0   26.0  
NORWAY              rid_108         87748.0   9.0   0.0   54.0  
                    rid_11         188183.0  20.0   1.0  131.0  
                    rid_36         208841.0  18.0   0.0  157.0

### 1.2. Unmonitored areas

In [4]:
# Read data
in_csv = f"/home/jovyan/shared/common/JES/teotil2_data/results/unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace("RENSEANLEGG", "sew") for i in umon_df.columns]
umon_df.columns = [i.replace("INDUSTRI", "ind") for i in umon_df.columns]
umon_df.columns = [i.replace("_tonn", "") for i in umon_df.columns]
umon_df.columns = [i.replace("AQUAKULTUR", "fish") for i in umon_df.columns]

# Convert Hg to kgs
umon_df["sew_Hg"] = umon_df["sew_Hg"] * 1000
umon_df["ind_Hg"] = umon_df["ind_Hg"] * 1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,424088.0,13169.0,1097.0,3006.0,285.0,68453.0,11825.0,35796.0,737.0,658.0,...,14740.0,0.0,0.0,0.0,4.0,12.0,2.0,0.0,2.0,1306.0
LOFOTEN-BARENTS SEA,131069.0,1496.0,163.0,678.0,84.0,18092.0,3134.0,6051.0,117.0,98.0,...,2240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,346.0
NORTH SEA,114941.0,3556.0,460.0,494.0,70.0,21118.0,3640.0,12828.0,209.0,276.0,...,5991.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,402.0
NORWEGIAN SEA2,170790.0,2884.0,388.0,939.0,95.0,29220.0,5046.0,14083.0,317.0,233.0,...,4647.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,557.0
SKAGERAK,7289.0,5232.0,87.0,896.0,36.0,23.0,4.0,2834.0,94.0,52.0,...,1862.0,0.0,0.0,0.0,2.0,7.0,1.0,0.0,1.0,0.0


### 1.3. Loads for 11 main rivers

In [5]:
# Read data
in_csv = f"/home/jovyan/shared/common/elveovervakingsprogrammet/results/measured_loads/loads_and_flows_all_sites_{year}.csv"
rid11_df = pd.read_csv(in_csv, index_col=0)
del rid11_df["new_rid_group"]

# Get data for RID11
rid11_df = rid11_df.query('old_rid_group == "rid_11"')

# Tidy
del rid11_df["station_code"], rid11_df["station_name"]
del rid11_df["old_rid_group"], rid11_df["ospar_region"]
del rid11_df["mean_q_1000m3/day"]

cols = [i for i in rid11_df.columns if i.split("_")[1] != "Est"]
rid11_df = rid11_df[cols]

# Convert units
rid11_df["Hg_kg"] = rid11_df["Hg_kg"] / 1000.0  # kg to tonnes
rid11_df["NH4-N_tonnes"] = rid11_df["NH4-N_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["NO3-N_tonnes"] = rid11_df["NO3-N_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["TOTN_tonnes"] = rid11_df["TOTN_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["TOTP_tonnes"] = rid11_df["TOTP_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["PO4-P_tonnes"] = rid11_df["PO4-P_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["SPM_tonnes"] = rid11_df["SPM_tonnes"] / 1000.0  # tonnes to ktonnes

# Tidy cols
rid11_df.columns = [i.split("_")[0] for i in rid11_df.columns]

rid11_df.head()

,Ag,As,Cd,Cr,Cu,Hg,NH4-N,NO3-N,Ni,PO4-P,Pb,SPM,SiO2,TOC,TOTN,TOTP,Zn
station_id,,,,,,,,,,,,,,,,,
29612,0.004673,0.869554,0.047343,3.211770,4.386634,0.007105,0.111180,2.249740,3.100457,0.017749,0.466015,16.227019,22147.614810,26435.660477,3.123075,0.045402,10.826743
29779,0.000000,0.502618,0.000000,0.856356,1.827911,0.003723,0.005260,0.161437,1.096341,0.059292,0.043117,290.226929,21938.881923,13904.064512,0.627370,0.091062,0.591784
29821,0.000000,0.201919,0.012808,0.154069,1.017809,0.002443,0.022878,0.269528,0.897434,0.001797,0.102857,2.147890,3390.149767,3934.109554,0.442282,0.008426,3.265497
29782,0.000000,0.836532,0.018213,1.593712,3.004602,0.007987,0.013794,0.217218,2.868342,0.008417,0.518629,22.618487,9308.093922,10321.797869,0.588850,0.020793,4.654334
36225,0.000236,0.009423,0.000693,0.009582,0.071448,0.000057,0.003860,0.025399,0.021026,0.002933,0.011138,1.262974,177.844927,237.172532,0.044173,0.003655,0.230223


## 2. Fill-in template

The template is usually sent each year by Csilla at NIBIO. However, it doesn't seem to change, so old versions can also be used e.g. here:

    ../../../../Results/OSPAR/Blank_Template

In [6]:
# Copy of template to update
temp_path = f"/home/jovyan/shared/common/elveovervakingsprogrammet/results/ospar/01_OSPAR_Norway_{year}.xlsx"

In [7]:
def update_spreadsheet_point_sources(xlsx, sheet, pars, src, df):
    """Update the OSPAR template for point source data.

    Args:
        xslx:  Str. Path to Excel template
        sheet: Str. Sheet name to update
        pars:  List. Parameter names in template to fill-in
        src:   Str. Type of input ('sew', 'ind', 'fish')
        df:    Dataframe. Values to fill-in

    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {
        "SPM": "S.P.M.",
        "TOC": "TOC",
        "PO4-P": "po4",
        "P-Total": "p",
        "NO3-N": "no3",
        "NH4-N": "nh4",
        "N-Total": "n",
        "As": "As",
        "Pb": "Pb",
        "Cd": "Cd",
        "Cu": "Cu",
        "Zn": "Zn",
        "Ni": "Ni",
        "Total Cr": "Cr",
        "Hg": "Hg",
    }

    # Map template names to df names
    names_dict = {
        "Norwegian Sea (NO)": "NORWEGIAN SEA2",
        "Barents Sea (NO)": "LOFOTEN-BARENTS SEA",
        "Skagerrak (NO)": "SKAGERAK",
        "North Sea (NO)": "NORTH SEA",
        "Norway Total": "NORWAY",
    }

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws["B12" : "B%s" % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws["E9":"AK9"][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        for par in pars:
            # Get value from df
            val = df.loc[names_dict[reg], "%s_%s" % (src, par_dict[par])]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

### 2.1. Sheet 5a: Sewage effluents

Note these are the values from Table 3 for **unmonitored areas** and not the total sewage inputs for each of the OSPAR areas. This is the same as what Tore reported previously.

In [8]:
# Get Sewage data
cols = [i for i in umon_df.columns if i.split("_")[0] == "sew"]
sew_df = umon_df[cols].copy()

# Convert units
sew_df["sew_Hg"] = sew_df["sew_Hg"] / 1000.0  # kg to tonnes
sew_df["sew_nh4"] = sew_df["sew_nh4"] / 1000.0  # tonnes to ktonnes
sew_df["sew_no3"] = sew_df["sew_no3"] / 1000.0  # tonnes to ktonnes
sew_df["sew_n"] = sew_df["sew_n"] / 1000.0  # tonnes to ktonnes
sew_df["sew_po4"] = sew_df["sew_po4"] / 1000.0  # tonnes to ktonnes
sew_df["sew_p"] = sew_df["sew_p"] / 1000.0  # tonnes to ktonnes
sew_df["sew_S.P.M."] = sew_df["sew_S.P.M."] / 1000.0  # tonnes to ktonnes

sew_df

,sew_n,sew_p,sew_po4,sew_no3,sew_nh4,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg
ospar_region,,,,,,,,,,,,,,
NORWAY,13.168960,1.097240,0.658344,0.658448,9.876720,14.740342,0.264510,0.273199,0.009284,4.329179,12.270663,1.667076,0.363571,0.001987
LOFOTEN-BARENTS SEA,1.496358,0.162642,0.097585,0.074818,1.122269,2.239801,0.007116,0.002744,0.000285,0.104476,0.232801,0.031176,0.018177,0.000100
NORTH SEA,3.556296,0.459660,0.275796,0.177815,2.667222,5.990726,0.087778,0.076235,0.002877,0.902668,3.316775,0.341907,0.100355,0.000486
NORWEGIAN SEA2,2.884155,0.387506,0.232504,0.144208,2.163116,4.647368,0.055655,0.051710,0.001536,1.384628,2.119620,0.178104,0.134046,0.000615
SKAGERAK,5.232151,0.087433,0.052460,0.261608,3.924113,1.862447,0.113961,0.142510,0.004586,1.937407,6.601467,1.115889,0.110993,0.000786


In [9]:
# Update sheet 5a
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
]

update_spreadsheet_point_sources(temp_path, "5a", pars, "sew", sew_df)

### 2.2. Sheet 5b: Industrial effluents

In [10]:
# Get industrial data
cols = [i for i in umon_df.columns if i.split("_")[0] == "ind"]
ind_df = umon_df[cols].copy()

# Convert units
ind_df["ind_Hg"] = ind_df["ind_Hg"] / 1000.0  # kg to tonnes
ind_df["ind_nh4"] = ind_df["ind_nh4"] / 1000.0  # tonnes to ktonnes
ind_df["ind_no3"] = ind_df["ind_no3"] / 1000.0  # tonnes to ktonnes
ind_df["ind_n"] = ind_df["ind_n"] / 1000.0  # tonnes to ktonnes
ind_df["ind_po4"] = ind_df["ind_po4"] / 1000.0  # tonnes to ktonnes
ind_df["ind_p"] = ind_df["ind_p"] / 1000.0  # tonnes to ktonnes
ind_df["ind_S.P.M."] = ind_df["ind_S.P.M."] / 1000.0  # tonnes to ktonnes

ind_df

,ind_n,ind_p,ind_po4,ind_no3,ind_nh4,ind_S.P.M.,ind_TOC,ind_As,ind_Pb,ind_Cd,ind_Cu,ind_Zn,ind_Ni,ind_Cr,ind_Hg
ospar_region,,,,,,,,,,,,,,,
NORWAY,3.006239,0.284663,0.170798,0.150312,2.254679,443.083824,1449.9514,1.896851,0.951793,0.064373,4.373590,12.192678,5.235763,1.182016,0.011505
LOFOTEN-BARENTS SEA,0.678003,0.083664,0.050198,0.033900,0.508502,216.660665,385.8789,0.000384,0.000004,0.000002,0.000170,0.003877,0.000972,0.000146,0.000003
NORTH SEA,0.493577,0.069931,0.041959,0.024679,0.370182,7.046103,689.7773,1.575322,0.750320,0.028722,0.529874,4.418339,2.813888,0.358255,0.003581
NORWEGIAN SEA2,0.938960,0.095470,0.057282,0.046948,0.704220,218.781459,267.5340,0.161981,0.072954,0.016914,0.890138,3.213748,0.289295,0.163329,0.004770
SKAGERAK,0.895699,0.035598,0.021359,0.044785,0.671774,0.595597,106.7612,0.159164,0.128515,0.018735,2.953408,4.556713,2.131607,0.660286,0.003151


In [11]:
# Update sheet 5b
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

update_spreadsheet_point_sources(temp_path, "5b", pars, "ind", ind_df)

### 2.3. Sheet 5c: Aquaculture discharges

In [12]:
# Get fish data
cols = [i for i in umon_df.columns if i.split("_")[0] == "fish"]
fish_df = umon_df[cols].copy()

# Convert units
fish_df["fish_nh4"] = fish_df["fish_nh4"] / 1000.0  # tonnes to ktonnes
fish_df["fish_no3"] = fish_df["fish_no3"] / 1000.0  # tonnes to ktonnes
fish_df["fish_n"] = fish_df["fish_n"] / 1000.0  # tonnes to ktonnes
fish_df["fish_po4"] = fish_df["fish_po4"] / 1000.0  # tonnes to ktonnes
fish_df["fish_p"] = fish_df["fish_p"] / 1000.0  # tonnes to ktonnes

fish_df

,fish_n,fish_p,fish_po4,fish_no3,fish_nh4,fish_Cu
ospar_region,,,,,,
NORWAY,68.452818,11.824837,8.159137,7.529810,54.762254,1305.754925
LOFOTEN-BARENTS SEA,18.092212,3.134393,2.162731,1.990143,14.473770,346.053433
NORTH SEA,21.117984,3.640444,2.511906,2.322978,16.894387,401.924032
NORWEGIAN SEA2,29.219866,5.046076,3.481792,3.214185,23.375893,557.344263
SKAGERAK,0.022756,0.003924,0.002707,0.002503,0.018205,0.433197


In [13]:
# Update sheet 5c
pars = ["NH4-N", "NO3-N", "P-Total", "PO4-P", "N-Total", "Cu"]

update_spreadsheet_point_sources(temp_path, "5c", pars, "fish", fish_df)

### 2.4. Sheet 5d: Other discharges

This sheet is left blank

### 2.5. Sheet 5e: Total direct discharges

The sum of sewage, industrial and fish-farm discharges.

In [14]:
# Combine sew, ind and fish, then aggregate
for df in [sew_df, ind_df, fish_df]:
    df.reset_index(inplace=True)
    df.columns = [i.split("_")[1] for i in df.columns]

td_df = pd.concat([sew_df, ind_df, fish_df], axis=0, sort=True)
td_df = td_df.groupby("region").sum()

td_df.columns = ["td_" + i for i in td_df.columns]

td_df

,td_As,td_Cd,td_Cr,td_Cu,td_Hg,td_Ni,td_Pb,td_S.P.M.,td_TOC,td_Zn,td_n,td_nh4,td_no3,td_p,td_po4
region,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,0.007500,0.000287,0.018323,346.158079,0.000103,0.032148,0.002748,218.900465,385.8789,0.236678,20.266573,16.104541,2.098861,3.380699,2.310515
NORTH SEA,1.663100,0.031599,0.458610,403.356574,0.004067,3.155795,0.826555,13.036828,689.7773,7.735114,25.167857,19.931792,2.525472,4.170035,2.829661
NORWAY,2.161361,0.073657,1.545587,1314.457694,0.013492,6.902839,1.224992,457.824165,1449.9514,24.463341,84.628016,66.893653,8.338570,13.206740,8.988279
NORWEGIAN SEA2,0.217636,0.018450,0.297375,559.619028,0.005385,0.467399,0.124664,223.428827,267.5340,5.333368,33.042980,26.243229,3.405341,5.529052,3.771578
SKAGERAK,0.273125,0.023321,0.771279,5.324012,0.003937,3.247496,0.271025,2.458044,106.7612,11.158180,6.150605,4.614092,0.308896,0.126954,0.076526


In [15]:
# Update sheet 5e
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

update_spreadsheet_point_sources(temp_path, "5e", pars, "td", td_df)

### 2.6. Sheet 6a: Monitored rivers

**Does "Inner Oslofjord" in the template correspond to "Alna"?** If so, I can fill-in one additional row in this table.

In [16]:
mon_df.reset_index(inplace=True)
tot_df = mon_df.groupby("ospar_region").sum()
trib_df = mon_df[mon_df["old_rid_group"] != "rid_11"].groupby("ospar_region").sum()

In [17]:
def update_spreadsheet_monitored_rivers(xlsx, sheet, pars, df_dict):
    """Update the OSPAR template for monitored rivers.

    Args:
        xslx:    Str. Path to Excel template
        sheet:   Str. Sheet name to update
        pars:    List. Parameter names in template to fill-in
        df_dict: Dict. {'tot':tot_df, 'trib':trib_df, 'main':rid11_df}
                 Values to fill-in

    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {
        "SPM": "SPM",
        "TOC": "TOC",
        "PO4-P": "PO4-P",
        "P-Total": "TOTP",
        "NO3-N": "NO3-N",
        "NH4-N": "NH4-N",
        "N-Total": "TOTN",
        "As": "As",
        "Pb": "Pb",
        "Cd": "Cd",
        "Cu": "Cu",
        "Zn": "Zn",
        "Ni": "Ni",
        "Total Cr": "Cr",
        "Hg": "Hg",
    }

    # Map template names to df names and rows
    names_dict = {
        "Norwegian Sea (NO)": ("NORWEGIAN SEA2", "tot"),
        "Barents Sea (NO)": ("LOFOTEN-BARENTS SEA", "tot"),
        "Skagerrak (NO)": ("SKAGERAK", "tot"),
        "North Sea (NO)": ("NORTH SEA", "tot"),
        "Norway Total": ("NORWAY", "tot"),
        "Tributary Rivers - Norwegian Sea": ("NORWEGIAN SEA2", "trib"),
        "Tributary Rivers - Barents Sea": ("LOFOTEN-BARENTS SEA", "trib"),
        "Tributary Rivers - Skagerak": ("SKAGERAK", "trib"),
        "Tributary Rivers - North Sea": ("NORTH SEA", "trib"),
        "Orkla": (29778, "main"),
        "Vefsna": (29782, "main"),
        "Alta": (29779, "main"),
        "Glomma": (29617, "main"),
        "Drammenselva": (29612, "main"),
        u"Numedalslågen": (29615, "main"),
        "Skienselva": (29613, "main"),
        "Otra": (29614, "main"),
        "Orreelva": (29783, "main"),
        "Vosso": (29821, "main"),
    }

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws["B12" : "B%s" % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws["E9":"AK9"][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        df_idx, df_name = names_dict[reg]

        # Get df
        df = df_dict[df_name]

        for par in pars:
            # Get value from df
            val = df.loc[df_idx, par_dict[par]]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

In [18]:
# Update sheet 6a
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]
df_dict = {"tot": tot_df, "main": rid11_df, "trib": trib_df}

update_spreadsheet_monitored_rivers(temp_path, "6a", pars, df_dict)

### 2.7. Sheet 6b: Unmonitored areas

In [19]:
# Get diff data
cols = [i for i in umon_df.columns if i.split("_")[0] == "diff"]
diff_df = umon_df[cols].copy()

# Convert units
diff_df["diff_nh4"] = diff_df["diff_nh4"] / 1000.0  # tonnes to ktonnes
diff_df["diff_no3"] = diff_df["diff_no3"] / 1000.0  # tonnes to ktonnes
diff_df["diff_n"] = diff_df["diff_n"] / 1000.0  # tonnes to ktonnes
diff_df["diff_po4"] = diff_df["diff_po4"] / 1000.0  # tonnes to ktonnes
diff_df["diff_p"] = diff_df["diff_p"] / 1000.0  # tonnes to ktonnes

diff_df

,diff_n,diff_p,diff_po4,diff_no3,diff_nh4
ospar_region,,,,,
NORWAY,35.795702,0.736925,0.181284,22.372314,1.968764
LOFOTEN-BARENTS SEA,6.050686,0.116505,0.028660,3.781679,0.332788
NORTH SEA,12.827649,0.208953,0.051402,8.017281,0.705521
NORWEGIAN SEA2,14.083377,0.317083,0.078002,8.802111,0.774586
SKAGERAK,2.833989,0.094385,0.023219,1.771243,0.155869


In [20]:
# Update sheet 6b
pars = ["NH4-N", "NO3-N", "P-Total", "PO4-P", "N-Total"]

update_spreadsheet_point_sources(temp_path, "6b", pars, "diff", diff_df)

### 2.8. Sheet 6c: Total inputs

**Note:** See e-mail from Csilla received 07/11/2017 at 13.43. This table should **not** include "point" discharges (`td_df`) - it's just the sum of tables 6a and 6b.

In [21]:
# Rename cols in diff_df
col_map = {
    "diff_n": "TOTN",
    "diff_p": "TOTP",
    "diff_po4": "PO4-P",
    "diff_no3": "NO3-N",
    "diff_nh4": "NH4-N",
}
diff_df.columns = [col_map[i] for i in diff_df.columns]

# Add to total_df
for col in diff_df.columns:
    tot_df[col] = tot_df[col] + diff_df[col]

# Update sheet 6c
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

df_dict = {"tot": tot_df, "main": rid11_df, "trib": trib_df}

update_spreadsheet_monitored_rivers(temp_path, "6c", pars, df_dict)

The following two cells are no longer used as I originally misunderstood what table 6c represents.

In [ ]:
## Standardise col names
# diff_df.columns = [i.split('_')[1] for i in diff_df.columns]
# td_df.columns = [i.split('_')[1] for i in td_df.columns]
# td_df.index.name = 'ospar_region'
#
# col_dict = {'SPM':'S.P.M.',
#            'TOTN':'n',
#            'NH4-N':'nh4',
#            'NO3-N':'no3',
#            'TOTP':'p',
#            'PO4-P':'po4'}
# for col in col_dict.keys():
#    new_col = col_dict[col]
#    tot_df[new_col] = tot_df[col]
#    del tot_df[col]
#
## Reset index
# diff_df.reset_index(inplace=True)
# tot_df.reset_index(inplace=True)
# td_df.reset_index(inplace=True)
#
## Concat and aggregate
##tot_df = pd.concat([diff_df, td_df, tot_df], axis=0).groupby('ospar_region').sum() # See comment above
# tot_df = pd.concat([diff_df, tot_df], axis=0).groupby('ospar_region').sum()
#
## Rename cols
# tot_df.columns = ['tot_'+i for i in tot_df.columns]
#
# tot_df

In [ ]:
## Update sheet 6c
# pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N',
#        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P',
#        'N-Total', 'Hg', 'Cu', 'TOC']
#
# update_spreadsheet_point_sources(temp_path, '6c', pars, 'tot', tot_df)

### 2.9. Sheet 7: Concentrations

In [23]:
# Read data
in_csv = f"/home/jovyan/shared/common/elveovervakingsprogrammet/results/measured_loads/concs_and_flows_rid_20_{year}.csv"
conc_df = pd.read_csv(in_csv, index_col=0, encoding="utf-8")

# Convert units
conc_df["Hg_ng/l"] = conc_df["Hg_ng/l"] / 1000  # ng to ug
conc_df["NH4-N_µg/l N"] = conc_df["NH4-N_µg/l N"] / 1000  # ug to mg
conc_df["NO3-N_µg/l N"] = conc_df["NO3-N_µg/l N"] / 1000  # ug to mg
conc_df["TOTN_µg/l N"] = conc_df["TOTN_µg/l N"] / 1000  # ug to mg
conc_df["TOTP_µg/l P"] = conc_df["TOTP_µg/l P"] / 1000  # ug to mg
conc_df["PO4-P_µg/l P"] = conc_df["PO4-P_µg/l P"] / 1000  # ug to mg
conc_df["TOC_mg C/l"] = conc_df["TOC_mg C/l"] * 1000  # mg to ug

# Get flags
cols = [i for i in conc_df.columns if i.split("_")[1] == "flag"]
lod_df = conc_df[cols]
lod_df.columns = [i.split("_")[0] for i in lod_df.columns]

# Get vals
cols = [
    i
    for i in conc_df.columns
    if ((i.split("_")[0] in lod_df.columns) and (i.split("_")[1] != "flag"))
]
conc_df = conc_df[cols]
conc_df.columns = [i.split("_")[0] for i in conc_df.columns]

# Rename
col_dict = {
    "SPM": "S.P.M.",
    "TOTN": "n",
    "NH4-N": "nh4",
    "NO3-N": "no3",
    "TOTP": "p",
    "PO4-P": "po4",
}
for col in col_dict.keys():
    new_col = col_dict[col]
    lod_df[new_col] = lod_df[col]
    conc_df[new_col] = conc_df[col]
    del conc_df[col], lod_df[col]

# Map Excel headings to df cols
par_dict = {
    "SPM": "S.P.M.",
    "TOC": "TOC",
    "PO4-P": "po4",
    "P-Total": "p",
    "NO3-N": "no3",
    "NH4-N": "nh4",
    "N-Total": "n",
    "As": "As",
    "Pb": "Pb",
    "Cd": "Cd",
    "Cu": "Cu",
    "Zn": "Zn",
    "Ni": "Ni",
    "Total Cr": "Cr",
    "Hg": "Hg",
}

# Map names to stns
names_dict = {
    "Orkla": 29778,
    "Vefsna": 29782,
    "Alta": 29779,
    "Glomma": 29617,
    "Drammenselva": 29612,
    "Numedalslågen": 29615,
    "Skienselva": 29613,
    "Otra": 29614,
    "Orreelva": 29783,
    "Vosso": 29821,
}

# Open new file and get sheet
wb = load_workbook(filename=temp_path)
ws = wb["7"]

# Get row numbers
row_dict = {}
for item in ws["B12" : "B%s" % ws.max_row]:
    # Get cell properties
    cell = item[0]
    name = cell.value
    row = cell.row
    row_dict[name] = row

# Get col numbers
col_dict = {}
for cell in ws["E9":"AK9"][0]:
    # Get cell properties
    par = cell.value
    col = cell.column
    col_dict[par] = col

pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

# Update spreadsheet
for reg in names_dict.keys():
    for par in pars:
        # Get values from df
        # 1a. Lower average
        vals = conc_df.loc[names_dict[reg], par_dict[par]].values.copy()
        lods = lod_df.loc[names_dict[reg], par_dict[par]].fillna("0").values.copy()
        vals[(lods == "<")] = 0
        val = np.nanmean(vals)

        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 1b. Upper average
        val = conc_df.loc[names_dict[reg], par_dict[par]].mean()
        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 2. Min
        val = conc_df.loc[names_dict[reg], par_dict[par]].min()
        row = row_dict[reg] + 2
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 3. Max
        val = conc_df.loc[names_dict[reg], par_dict[par]].max()
        row = row_dict[reg] + 3
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 4. N
        n_samp = len(conc_df.loc[names_dict[reg], par_dict[par]].dropna())
        row = row_dict[reg] + 5
        col = col_dict[par]
        ws.cell(row=row, column=col).value = n_samp

        # 5. N LOD
        n_lod = (~pd.isnull(lod_df.loc[names_dict[reg], par_dict[par]])).sum()
        pct_lod = 100 * float(n_lod) / float(n_samp)

        if pct_lod < 30:
            val = "Yes"
        else:
            val = "No"

        row = row_dict[reg] + 4
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 6. Std
        val = conc_df.loc[names_dict[reg], par_dict[par]].std()
        row = row_dict[reg] + 7
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

# Save
wb.save(temp_path)

### 2.10. Sheet 9: Discharge

The notebook [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/recalculate_ospar_flows.ipynb) handles the OSPAR flow data. Run this, and then copy the results over to Sheet 9. 